<div class="alert alert-block alert-info">

<b>Thank you for contributing to TeachOpenCADD!</b>

</div>

<div class="alert alert-block alert-info">

<b>Set up your PR</b>: Please check out our <a href="https://github.com/volkamerlab/teachopencadd/issues/41">issue</a> on how to set up a PR for new talktorials, including standard checks and TODOs.

</div>

# · Diffusion-based docking models

**Note:** This talktorial is a part of TeachOpenCADD, a platform that aims to teach domain-specific skills and to provide pipeline templates as starting points for research projects.

Authors:

- Hamza Ibrahim, CADD seminars 2023, Universität des Saarlandes (UdS)
- Michael Bockenköhler, 2023,  [Volkamer lab](https://volkamerlab.org), Universität des Saarlandes (UdS)
- Andrea Volkamer, 2023,  [Volkamer lab](https://volkamerlab.org), Universität des Saarlandes (UdS)

## Aim of this talktorial

This talktorial presents new class of generative models which called diffusion probabilistic model. You will learn what diffusion models are and how it can be applied on molecular docking.

### Contents in *Theory*


* Diffusion generative models (DGM).
    1. Forward process
    2. Reverse process
    3. Train a diffusion model
        - Loss function
        - Network architecture
* Diffusion-based docking models.
    1. Ligand pose manifold
    2. Product space diffusion
    3. Training 
    4. inference

### Contents in *Practical*

* Data preparation.
    - Download PDB structure
    - Prepare input file
* DiffDock implementation
    

### References

* Denoising Diffusion Probabilistic Models: [<i>arXiv</i> (2020)](https://arxiv.org/pdf/2006.11239.pdf?ref=assemblyai.com)
* Score-based generative modeling through stochastic differential equations: [<i>arXiv</i> (2021)](https://arxiv.org/pdf/2011.13456.pdf) 
* Equivariant Graph Neural Networks: [<i>arXiv</i> (2022)](https://arxiv.org/pdf/2102.09844.pdf)
* Structure-based Drug Design with Equivariant Diffusion Models: [<i>arXiv</i> (2022)](https://arxiv.org/pdf/2210.13695.pdf)
* DiffDock: Diffusion Steps, Twists, and Turns for Molecular Docking: [<i>arXiv</i> (2023)](https://arxiv.org/pdf/2210.01776v2.pdf)
* [Diffusion Model Clearly Explained!](https://medium.com/@steinsfu/diffusion-model-clearly-explained-cd331bd41166)
* Deep Unsupervised Learning using Nonequilibrium Thermodynamics: [Sohl-Dickstein et al. <i>arXiv</i> (2021)](https://arxiv.org/pdf/1503.03585.pdf)
* Generative Modeling by Estimating Gradients of the Data Distribution: [Song et al. <i>arXiv</i> (2019)](https://arxiv.org/abs/1907.05600)
* Denoising Diffusion Probabilistic Models [Ho et al. <i>arXiv</i> (2020)](https://arxiv.org/abs/2006.11239)

## Theory

### Diffusion generative models (DGM).

Generative models are a category of machine learning models that have the capability to generate new data based on a given training data set. Diffusion probabilistic model (DPM) is a type of generative models which is inspired from Physics by non-equilibrium thermodynamics[ref]. DPM depends on two main reciprocal processes that represent two sets of random variables organized in the form of Markov chains.


1. Forward Diffusion Process → add noise to input data.
2. Reverse Diffusion Process → denoise noised data.

![DGM processes figure](images/basics_dgm.png)

*Figure 1:* 
Black arrows represent the forward diffusion process, while blue arrow represents the reverse diffusion process
Figure is taken from: [Medium article](https://medium.com/@steinsfu/diffusion-model-clearly-explained-cd331bd41166).

#### 1. Forward process

The first process adds guassian noise sequentially to the input data $x_0$ by $T$ steps. As $T → \infty $, $x_T$ becomes a complete static noise image as in figure [1]. So every successive state 
$\mathbb{x}_{t + 1}$ could be computed as the following : 
$$
q(\mathbb{x}_{t}|{x}_{t-1}) = \mathcal{N(\mathbb{x}_{t};\mathbb{\mu}_t = \sqrt{1 - \beta}{x}_{t-1}, \Sigma_t = \beta_t \mathbf{I})},
$$
Where $q(\mathbb{x}_{t}|{x}_{t-1})$ denotes the distribution of the next state $\mathbb{x}_{t}$.

 $\mathbb{\mu}_t$ and $\Sigma_t({x}_t, t)$ represent the mean and covariance of next state distribution, respectively.

Utilizing [Reparametrization Trick](https://medium.com/@steinsfu/diffusion-model-clearly-explained-cd331bd41166#228f) closed-Form formula could be derived, which prompts us to sample ${x}_{t}$ at any time step using ${x}_{0}$. It makes forward diffusion process much faster as following:
$$
{x}_{t} = \sqrt{{\u\alpha}_t} {x}_0 + \sqrt{1 - {\u\alpha}_t} {\epsilon}_0,
$$

Where ${\u\alpha}_t = \prod_{s = 0}^{t}{1 - {\beta}_s}$ , and ${\epsilon}_0, ... , {\epsilon}_{t-2}, {\epsilon}_{t-1} \sim \mathcal{N (0 , \mathbf{I})}$

#### 2. Reverse process

Unfortunetaly, It's not possible to sample ${x}_{0}$ from ${x}_{t}$ using $q(\mathbb{x}_{t-1}|{x}_{t})$ as in forward process, because reversing the noise is intractable, therefore **reverse diffusion process** is employed. As a solution $q(\mathbb{x}_{t-1}|{x}_{t})$ could be approximated by using a deep learning model (e.g. neural network), which predicts an approximation to the conditional probability distribution $\mathbb{p}_{\theta}(\mathbb{x}_{t-1}|{x}_{t})$, which modeled as a Gaussian distribution:

$$
\mathbb{p}_{\theta}(\mathbb{x}_{t-1}|{x}_{t}) = \mathcal{N(\mathbb{x}_{t-1};\mathbb{\mu}_\theta({x}_t, t), \Sigma_t({x}_t, t))},
$$

By learning the conditional probability densities using deep learning model the original image $x_0$ is reconstructed from the noisy image $\mathbb{x}_t$ as illustrated in _figure 1_. Allowing for the extraction of meaningful information from the noisy representation.



After explaining the two main processes of diffusion models, we start now with training the model.

#### 3. Train a diffusion model

In order to effectively train model, It's necessary to define an optimized loss function and the architecture of the deep learning model. In this section we will explain briefly the loss function and the network archietecture, which commonly employed in diffusion models then we'll explain the training process of diffusion models.

##### - Loss function

Usually the loss function is the difference between predicted and true values. In case of DGMs the true value corresponds to the added noise that introduced to an image and the model's objective is to predict the added noise. [Ho et al. (2020)](https://arxiv.org/pdf/2006.11239.pdf) has simplified the loss function to:
$$
{L}_{t}^{simple} = \mathbb{E}_{x_0, t, \epsilon}[|| \epsilon - {\epsilon}_\theta(\sqrt{a^u} x_0 + \sqrt{1 - a^u} \epsilon,t)||^2]
$$ 
where: 

$\epsilon \sim \mathcal{N}(0, \mathbb{I})$ is the actual noise added, whch follows a standard normal distribution.

${\epsilon}_\theta(\sqrt{\u\alpha} x_0 + \sqrt{1 - \u\alpha} \epsilon,t) = {\epsilon}_\theta(x_t,t) $ denotes the approximated noise from neural network using reparamarization trick that mentioned before.

As observed, the loss function is the mean square error (MSE) of the added noise and predicted noise.

Once the loss function has been chosen, we can go to the next step, which is selecting an appropriate network architecture and training the diffusion model.

##### - Network architecture

The most important requirment of the network is to have the identical dimensionality for the input and the output. Therefore, usually [U-Net](https://theaisummer.com/unet-architectures/) is commonly used for prediction tasks in DGMs as a network architecture.

The U-Net architecture based on an encoder-decoder structure. In encoding, the spatial dimensions decreases while the number of channels increases keeping the important features of the input data. On the contrary, in decoding the spatial dimensions increase while number of channels decrease to produce the same spatial dimensions as the input data as illustrated in Figure 2.


<img src="images/Unet-architecture.png"  style="margin-left: auto; margin-right: auto;">

*Figure 2:* 
An overview of U-Net architecture.
Figure is taken from: [AI Summer](https://theaisummer.com/static/fa507fda71846a516801bccb19474aec/0012b/Unet-architecture.png).

After choosing the network architecture and the loss function, we can train the model and as we explained. For more information on diffusion model training, you can check [this article](https://medium.com/@steinsfu/diffusion-model-clearly-explained-cd331bd41166#02b8).

Now we got a clear idea of the outline diffusion models. In the next section, we'll explain the diffusion models used in the context of molecular docking problems.

### Diffusion-based docking model.

The main concepts of creating a diffusion models are explained. It's important to note that implementing DGMs in molecular docking will not be the same as we explained. In this section we discuss the challenges encountered in applying DGMs in molecular docking and how these obstacles have been overcome in a real-case application.

#### 1. Ligand pose manifold

In order to have diffusion-based docking model, you have to think of manifold that suits ligand poses where $L \isin \mathbb{R}^{3n} $ as $n$ is the number of atoms. If we start forward diffusing without setting any limitations for the degree of freedom, it becomes absurd and ligands will have unreasonable bond lengths and angles as in figure 3.

<img src="images/absurd_ligand.png"  style="margin-left: auto; margin-right: auto;">

*Figure 3:* 
Randomizing bond length and angles without keeping local structures fixed.

A solution to this problem is presented in [DiffDock paper](https://arxiv.org/pdf/2210.01776v2.pdf). They are inspired from traditional docking approches by taking already embedded ligand in a 3D space using RDKit , which instantiates the angles and bond length of the atoms. Instead of thinking of a ligand as an element in an eucledian space, they described ligand pose by four main parameters. 

1. Local structures like bond lengths, bond angles, chirality and ring structure are generated using RDKit and kept fixed in order to maintain integrity of the model.

2. Position of ligand with 3D translastion group, where left flexible to find the pocket and fit in it $\mathbb{R}^3$.

3. Rotation parameterization, where $Rotation \isin {SO(3)}$ correspnds to 3D rigid rotation around the mass centre of the ligand.

<img src="images/rotation.gif"  style="width:300px;">


*Figure 4:* 
GIF shows an example of the rotaion of a methyl group in an Ethane structure. Figure taken from [Proteopedia](https://proteopedia.org/wiki/index.php/Dihedral/Index)

4. Flexibility of torsion angles to fit in the pocket, where: $ \mathit{Torsions} \isin \mathbb{T}^m$, which represent the changes in torsion angles around rotatable bonds in a ligand with a copy of 2D rotation group ${SO(2)}$ .  

<img src="images/Phipsi-AH.gif"  style="width:400px;">

*Figure 5:* 
GIF illustrartes the torsion angles and changes in it. As shown a torsion angle ϕ is defined by  a four covalently bonded atoms. Every three atoms defines a half plane and when these planes intersect the angle between them is torsion angle ϕ. Figure taken from [Proteopedia](https://proteopedia.org/wiki/index.php/Dihedral/Index)

These four parameters have introduced a new challenge. The problem arises from the fact that there are several valid possibilities for making changes through rotations and alterations in torsion angles together. The used strategy in DiffDock is to _disentangle_ the degrees of freedom involved in docking, which aims to isolate the modifcation of torsion angles from other transformations such as rotation and translations.

To make sure that the changes in torsion was totally independent during docking, post-torsion RMSD alignment was performed to confirm that rotations and translations were orthogonal to torsion modifications.

By utilizing those parameters, it was possible to map ligand poses into submanifold $M_c \subset \mathbb{R}^{3n}$, where they can easily diffuse over. This submanifold $M_c$ facilitates diffusion over a space where ligand poses are represented in $(m + 6)$ dimensions, where $m$ denotes number of rotatable bonds.

Fortunately, the ligand pose submanifold establishes a smooth mapping with the product space. As a result, we can now map displacements within the manifold of ligand poses to the product space. This product space, denoted as : $\mathbb{P} = \mathbb{R}^3 * \isin {SO(3)} * \isin \mathbb{T}^m$

TODO

- Score matching abstract idea 
- Product space diffusion
- confidence interval
* practical
- More explaining
- visualize true ligand
- visualize predicted ligand
- Calculate RMSD between them if applicable
- PDB reverse visualization (GIF generation if applicable)


#### 2. Product space diffusion

After mapping ligand pose manifold to product space, we need to figure out how the score model will diffuse on the product space. Most of existing score-based generative models are designed as if the data is on a eucalidean space, as if it's a flat geometry.[De Bortoli et al. 2022](https://arxiv.org/pdf/2202.02763.pdf) developed Riemannian score-based generative model (SGM) which based on Riemannian manifold which gives the possibility to create SGMs of a various manifolds.

The main concept is to consider the score model not as vector field on the eucledian space,but rather as a vector field on the manifold where score and the score model are elements of the tangent space of every possible point on the manifold as represented in _figure 6_. 

<img src="images/tangent_space.png"  style="width:400px;">

*Figure 6:* The tangent space, denoted as ${T_xM}$ represents the set of all possible tangent vectors $v$ at $x$ as $x \isin \mathbb{M}$. Where $\mathbb{M}$ is, in our case, the product manifold $\mathbb{P}$.

A standard score model is used to be trained according to [Song et al. 2019](https://arxiv.org/abs/1907.05600). That leads to simplifying the task at hand and we now need only to build a methods for sampling from the diffusion kernel on the product space and computing the score.

As mentioned before Product space is the product of three manifolds.So, in order to proceed the forward diffusion process on the product space, every manifold will be diffused independently according to [Rodol`a et al., 2019](https://arxiv.org/abs/1809.10940) and the tangent space will become a direct sum of every manifold:

$$
T_g \mathbb{P} = T_r\mathbb{T^3} \oplus T_RSO(3) \oplus T_\theta SO(2)^m 
$$

Now, we can diffuse over the product space by sampling from the diffusion kernel and independently regressing the score within each group.





Standard score matching is used to train the model. Now the only need method is to sample from and compute the score of the diffusion kernel on $\mathbb{P}$.

Starting with forward diffusion process. Forward diffusion computed of the direct sum of tangent space for each manifold independently since $\mathbb{P}$ is a product space. Forward stochastic differential equation (SDE) is defined as the follow:
$$
dx = \sqrt{d \sigma^2(t)/dt}dw
$$
Where $\sigma^2 = \sigma^2_{pos} ,  \sigma^2_{rot} ,  \sigma^2_{tr} $ and $w$ indicates [brownian motion](https://en.wikipedia.org/wiki/Brownian_motion).

##### Confidence score abstract idea

![ChEMBL web service schema](images/DiffDock.png)

*Figure 2:* 
Overview of DiffDock. Left: The model takes as input the separate ligand and protein
structures. Center: Randomly sampled initial poses are denoised via a reverse diffusion over trans-
lational, rotational, and torsional degrees of freedom. Right:. The sampled poses are ranked by the
confidence model to produce a final prediction and confidence score.
Figure and discription taken from: [arXiv 2023](https://arxiv.org/pdf/2210.01776v2.pdf).

## Practical

We will now proceed to implement software used in molecular docking trained using diffusion generative model called DiffDock. It's now open-source and published on [Github](https://github.com/gcorso/DiffDock).

#### Import dependancies

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import os 
import nglview as nv
import urllib
from pathlib import Path

In [2]:
from pathlib import Path
HERE = Path(_dh[-1])
DATA = HERE / "data"

### Data preparation

Before starting DiffDock implementation, input data has to be prepared. First we have to 

#### Dwonload PDB structure

In [3]:
# Specify the PDB code of the structure you want to use
# P.S. you can use your own protein, but you need to place it in DATA

protein_pdb = '6w70.pdb'

In [4]:
# Download the PDB structure if it's not in DATA

if protein_pdb not in os.listdir('data'):
    urllib.request.urlretrieve(f'http://files.rcsb.org/download/{protein_pdb}', f'data/{protein_pdb}')
else:
    print('PDB structure already downloaded.')

PDB structure already downloaded.


#### Specify your quary ligand

In [4]:
# Give ligand SMILES here
molecule_id = ['Molecule_1']
ligand_smiles = ["O=C(O)c1cc(/OCc2cccc3ccccc23)ccc1O"]

In case of mutliple ligands wanted to be docked on the same protein, this function creates a csv file that can be used as an input and get the results.

### DiffDock implementation

First step is to download DiffDock software from [Github repository](https://github.com/gcorso/DiffDock.git).

In [5]:
if "DiffDock" not in os.listdir(HERE):
    #specifiy version
    !git clone https://github.com/gcorso/DiffDock.git
else:
    print(f"DiffDock is alreay cloned.")

DiffDock is alreay cloned.


#### Configure your docking settings 


In [7]:
samples_per_complex = "5"
inference_steps = "10"
actual_steps = "18"
batch_size = "5"
print(f'{DATA}/{protein_pdb}')

/home/hamza/Desktop/Bioinformatics_master/SS23/CADDSeminar_2023/notebook/T02_DiffusionBasedDocking/data/6w70.pdb


In [9]:
os.chdir("DiffDock/")
for id, smiles in zip(molecule_id, ligand_smiles):
    diffdock_cmd = f"python -m inference --protein_path {DATA}/{protein_pdb} --ligand '{smiles}' --out_dir {DATA}/{id} --inference_steps {inference_steps} --samples_per_complex {samples_per_complex} --save_visualisation --batch_size {batch_size} --actual_steps {actual_steps} --no_final_step_noise"
    os.system(diffdock_cmd)
os.chdir("../")

Generating ESM language model embeddings


/home/hamza/mambaforge-pypy3/envs/diffdock/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6003.
  warnings.warn(
/home/hamza/mambaforge-pypy3/envs/diffdock/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6128.
  warnings.warn(
/home/hamza/mambaforge-pypy3/envs/diffdock/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6221.
  warnings.warn(
/home/hamza/mambaforge-pypy3/envs/diffdock/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6381.
  warnings.warn(


Processing 1 of 1 batches (2 sequences)
HAPPENING | confidence model uses different type of graphs than the score model. Loading (or creating if not existing) the data for the confidence model now.
Size of test dataset:  1


0it [00:00, ?it/s]

Failed on ['1a0q'] index 10 is out of bounds for axis 0 with size 10
Failed for 1 complexes <torch_geometric.loader.dataloader.DataLoader object at 0x7fbc0b37c160>
Skipped 0 complexes
Results are in /home/hamza/Desktop/Bioinformatics_master/SS23/CADDSeminar_2023/notebook/T02_DiffusionBasedDocking/data/Molecule_1


1it [01:05, 65.16s/it]


In [7]:
import nglview as nv

view = nv.show_pdbid(f"{DATA}/{id}/rank1_reverseprocess.pdb")  # Or use nv.show_file("path/to/your/file.pdb")

# Display the molecular viewer
display(view)

InvalidURL: URL can't contain control characters. '/pdb/files//home/hamza/Desktop/Bioinformatics_master/SS23/CADDSeminar_2023/notebook/T02_DiffusionBasedDocking/data/<built-in function id>/rank1_reverseprocess.pdb.cif' (found at least ' ')

In [2]:
import py3Dmol
from rdkit import Chem

# Read the PDB file
with open(f'{DATA}/{protein_pdb}', 'r') as pdb_file:
    pdb_data = pdb_file.read()

# Read the SDF file
suppl = Chem.SDMolSupplier(f'{DATA}/results/Molecule_1/1a0q/rank1.sdf')
print(suppl)
# Create a viewer
viewer = py3Dmol.view(width=800, height=600)

# Add the PDB data to the viewer
viewer.addModel(pdb_data, 'pdb')

# Iterate over the molecules in the SDF file
for mol in suppl:
    # Skip invalid molecules

    if mol is None:
        continue

    # Add the SDF molecule to the viewer
    viewer.addModel(Chem.MolToMolBlock(mol), 'sdf')

# Set the style and visualization options
# viewer.addModel(Chem.MolToMolBlock(suppl), 'sdf')
viewer.setStyle({'cartoon': {'color': 'spectrum'}})
viewer.zoomTo()

# Display the viewer
viewer.show()

ModuleNotFoundError: No module named 'py3Dmol'

## Discussion

Wrap up the talktorial's content here and discuss pros/cons and open questions/challenges.

## Quiz

Ask three questions that the user should be able to answer after doing this talktorial. Choose important take-aways from this talktorial for your questions.

1. Question
2. Question
3. Question

<div class="alert alert-block alert-info">

<b>Useful checks at the end</b>: 
    
<ul>
<li>Clear output and rerun your complete notebook. Does it finish without errors?</li>
<li>Check if your talktorial's runtime is as excepted. If not, try to find out which step(s) take unexpectedly long.</li>
<li>Flag code cells with <code># NBVAL_CHECK_OUTPUT</code> that have deterministic output and should be tested within our Continuous Integration (CI) framework.</li>
</ul>

</div>